# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. This is the dataset to use for this assignment. Note: The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [1]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd
import numpy as np          
from matplotlib import dates, pyplot

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np           
import matplotlib.dates as mdates

df = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')

df['Data_Value'] = df['Data_Value']*0.1

df['Date'] = pd.to_datetime(df['Date'])

df['Year'] = df['Date'].dt.year

df['Date'] = df['Date'].dt.strftime('%m %d')

df = df[df['Date']!='02 29']

maxT = df[(df.Year >= 2005) & (df.Year < 2015) & (df['Element'] == 'TMAX')].groupby(['Date'])['Data_Value'].max().to_frame().rename(columns={'Data_Value':'MaxT'})
minT = df[(df.Year >= 2005) & (df.Year < 2015) & (df['Element'] == 'TMIN')].groupby(['Date'])['Data_Value'].min().to_frame().rename(columns={'Data_Value':'MinT'})
data = maxT.merge(minT, left_index = True, right_index= True, how = 'outer').reset_index()

df = df.merge(maxT.reset_index(drop=False), on='Date', how='left')
df = df.merge(minT.reset_index(drop=False), on='Date', how='left')
df = df.drop(['ID'], axis =1).sort_values(by = 'Date')
record_max_2015 = df[(df.Year==2015)&(df['Element'] == 'TMAX')&(df.Data_Value > df.MaxT)].rename(columns={'Data_Value':'MaxT_2015'}).groupby(['Date'])['MaxT_2015'].max().to_frame()
record_min_2015 = df[(df.Year==2015)&(df['Element'] == 'TMIN')&(df.Data_Value < df.MinT)].rename(columns={'Data_Value':'MinT_2015'}).groupby(['Date'])['MinT_2015'].min().to_frame()
data_2015 = record_max_2015.merge(record_min_2015, left_index = True, right_index= True, how = 'outer').reset_index()


data['Date'] = pd.to_datetime(data['Date'], format = '%m %d')

data_2015['Date'] = pd.to_datetime(data_2015['Date'], format = '%m %d')

data = data.merge(data_2015,left_on = 'Date', right_on= 'Date', how = 'outer').reset_index()

plt.figure(figsize=(15,10))
plt.scatter(x=data['index'], y=data['MaxT_2015'],color='blue')
plt.scatter(x=data['index'], y=data['MinT_2015'],color='red')
plt.plot(data['index'], data['MaxT'], linewidth = 0.5,color='blue')
plt.plot(data['index'], data['MinT'], linewidth = 0.5,color='red')
plt.gca().fill_between(data['index'], data['MaxT'], data['MinT'], facecolor ='grey', alpha = 0.2)

plt.xlabel('Months', fontsize = 18, labelpad=20)
plt.ylabel('Temperature (°C)',fontsize = 18)
plt.title('Record high and record low temperatures by day of the year over the period 2005-2015 near Ann Arbor, Michigan, United States', fontsize = 15)
plt.legend(['Max_temp(2005-2014)', 'Min_temp(2005-2014)','Max_temp(2015)','Min_temp(2015)'],loc=1, frameon=False, title='Legend')
plt.xticks(np.linspace(0,364,13)[:-1], ('Jan', 'Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov', 'Dec'), fontsize = 15)
plt.yticks(fontsize = 15)

for spine in plt.gca().spines.values():
    spine.set_visible(False)

plt.savefig('output.pdf')
plt.show()


FileNotFoundError: [Errno 2] File data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv does not exist: 'data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv'

In [28]:
#import calendar
#a = np.arange(0,364,1)
#b = pd.Series(pd.to_datetime(a,unit='D')).dt.month.to_frame().rename(columns={0:'Month'})
#b['Month'] = b['Month'].apply(lambda x: calendar.month_abbr[x])

#plt.gca().fill_between(data['Date'].dt.to_pydatetime(), data['MaxT'], data['MinT'], facecolor ='yellow', alpha = 0.4)

#df_min['MonthDay'] = df_min['Date'].apply(lambda x: x.strftime("%B %d") if pd.notnull(x) else '')
#df_max['MonthDay'] = df_max['Date'].apply(lambda x: x.strftime("%B %d")

SyntaxError: unexpected EOF while parsing (<ipython-input-28-533269507006>, line 8)